In [4]:
import sys
sys.path.insert(1, './lib')
import time
import csv_io
import ontoinfo
import gitcrawl
from github import Github
from github import Auth
from pygit2 import Repository
import glob
import re

from rdflib import BNode, URIRef, Literal, Graph, Namespace
from rdflib.collection import Collection
from rdflib.util import guess_format
from rdflib.namespace import RDF, XSD, RDFS, OWL, SKOS, DCTERMS, NamespaceManager
from rdflib.plugins.sparql import prepareQuery
from datetime import datetime
from urllib.request import urlopen, pathname2url
from urllib.parse import urlparse, urljoin
from typing import Dict, List, Tuple

In [2]:
auth = Auth.Token("YOUR TOKEN")
git = Github(auth=auth)

In [5]:
allq  = prepareQuery("SELECT ?s ?p ?o WHERE {?s ?p ?o}")

filename="mse_ontologies.ttl"
this_ontology_url=ontoinfo.path2url(filename)
emtest=Graph()
emtest.parse(this_ontology_url)
#pref = 'http://example/oekg/'
#qres = emtest.query(classes)
#names=[]
#for row in qres:
#    names.append(row.entity.strip()[len(pref):])
#    #print(row)

qres = emtest.query(allq)
links=[]
for row in qres:
    if ("resourceLocator" in row.p):
        links.append(row.o.strip())

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Yamir\\Documents\\GitHub\\OntoWalker\\Ebr\\mse_ontologies.ttl'

In [3]:
user="OCDO"
folder="test2"

In [6]:
#OUTPUT PATHs
outname = "Output/GitTest.csv"
csv_io.create_csv(outname)

In [13]:
def get_ai_model():
    from openai import AzureOpenAI
    ai_client = AzureOpenAI(
      azure_endpoint = "https://my-east.openai.azure.com/", 
      api_key="YOUR API KEY",  
      api_version="2023-05-15"
    )

    #modelName="Microsoft.CognitiveServicesOpenAI-20240111194907"
    #deployment_name='gpt-35'
    return ai_client

try:
    my_ai = get_ai_model()
    start_phrase = "say hello"
    
    response = my_ai.chat.completions.create(
        model="gpt-35", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": start_phrase}
        ]
    )
    #print(response.choices[0].message.content.format(1))
    print("AI connected")
except Exception as e:
    print("An error occurred:", e)
    my_ai = []

An error occurred: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}


In [5]:
repo_list = gitcrawl.get_user_repos(user,git)

['.github', 'asmo', 'cdco', 'cdos', 'cmso-ontology', 'DSIM', 'ldo', 'mmss', 'pldo', 'podo']


In [6]:
for repo in repo_list:
        time.sleep(1)
        print("NEXT REPO")
        prefix="https://github.com"
        subfold="/"+user+"/"+repo
        print("link=" + prefix + subfold)
        repath = folder+subfold 
        try:
            clone_repo(folder,user,repo)
        except:
            print("repo exists?")
            pass
            
        try:
            repobj = Repository(repath)
        except:
             print("repo failed to found?")
             continue

        #INFO FROM REPOSITORY ITSELF
        [lastdate,inidate] = gitcrawl.get_first_last_commit(repobj)   #first commit date

        if (my_ai!=[]):
            read_info = gitcrawl.extract_readme(repath,my_ai)        #parse README.md if available (with AI currently)
        else:
            read_info = ["no info","no info","no info"]       #otherwise no info - contact point, related project, documentation link
    
        lic_info = gitcrawl.extract_license(repath)        #get first line of the LICENSE file
    
        #looking for onto files
        onto_infos=[]
        scores=[]
        maxscore=-1
        for ext in ontoinfo.ext_list:  #check all ontology extensions
                files = glob.glob(repath + '/**/*.' + ext, recursive=True)
                for fi in files:
                    success = ontoinfo.querry_successful(fi)  #test querryability of the file
                    if (success):

                        #LINK TO THE FILE (default branch)
                        #x=fi[len(folder):].replace("\\","/")
                        #ind=[x.start() for x in re.finditer('/',x)][2]
                        #branch = Repository(folder + x[:ind]).head.shorthand
                        #gitlink = x[:ind] + "/tree/" + branch + x[ind:]
                        #print(gitlink)
                        
                        # LINK TO REPOSITORY
                        gitlink = prefix + subfold

                        if (gitlink in links):
                            print("ontology already present!")
                        else:
                            links.append(gitlink)
                            
                        #extract information from ontology file
                        oinfo_fill = ontoinfo.extract_info(fi,gitlink)   

                        #add to the csv file
                        csv_io.add_to_csv(outname,oinfo_fill[0],oinfo_fill[1],oinfo_fill[2],oinfo_fill[3],oinfo_fill[4],lic_info,read_info[1],read_info[0],read_info[2],inidate,lastdate)

NameError: name 'repo_list' is not defined

In [7]:
[lastdate,inidate]  = [2,3]

In [8]:
lastdate

2